In [40]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import pandas as pd

In [3]:
# scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased") # 
# scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

In [1]:
import torch

In [64]:
from transformers import BertModel, BertTokenizer

# Retrieve SciBERT.
scibert_model = BertModel.from_pretrained("allenai/scibert_scivocab_uncased",
                                  output_hidden_states=True)
scibert_tokenizer = BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

scibert_model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31090, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [2]:
from transformers import BertTokenizer, BertModel

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [18]:
text = "Hydrogels are hydrophilic polymer networks which may absorb from " \
       "10–20% (an arbitrary lower limit) up to thousands of times their " \
       "dry weight in water."

In [67]:
input_id = bert_tokenizer(pone['Total'][6],  return_tensors="pt",  add_special_tokens = True)['input_ids'][:,:512]

Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors


In [68]:
len(input_id[0])

512

In [49]:
outputs = bert_model(input_id.to('cuda:0'))

In [50]:
token_vecs = outputs[0]

In [51]:
token_vecs

tensor([[[-0.5995, -0.2517, -0.3147,  ...,  0.1006,  0.4913,  0.4781],
         [-0.4720, -0.3171, -0.1817,  ...,  0.2245,  1.1088,  0.8908],
         [-0.1618, -0.5751,  1.4681,  ...,  0.2190, -0.3893,  1.0191],
         ...,
         [-0.3682,  0.2389,  0.3278,  ..., -1.3229, -0.1476,  0.2587],
         [-0.6185, -0.4585,  0.2965,  ..., -0.4635, -0.0534, -0.4641],
         [-0.6276, -0.2235,  0.1513,  ...,  0.4784, -0.4683, -0.0689]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>)

In [4]:
import torch

In [5]:
def get_scibert_embedding(text):
    
    input_id = scibert_tokenizer(text,  return_tensors="pt",  add_special_tokens = True)['input_ids'][:,:512]
    
    with torch.no_grad():
        outputs = scibert_model(input_id.to('cuda:0'))
        hidden_states = outputs[2]
        token_vecs = hidden_states[-2]

    return input_id.numpy(), token_vecs.cpu().numpy()


In [15]:

def get_bert_embedding(text):
    # Encode the text, adding the (required!) special tokens, and converting to
    # PyTorch tensors.
    input_id = bert_tokenizer(text,  return_tensors="pt",  add_special_tokens = True)['input_ids'][:,:512]
#     print(len(input_id[0]))
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():
        outputs = bert_model(input_id.to('cuda:0'))
        token_vecs = outputs[0]
        
    return input_id.numpy(), token_vecs.cpu().numpy()


In [7]:
import pandas as pd

In [8]:
pone = pd.read_csv('../../Data/Experiment/Plosone/pone_final.csv',index_col='Unnamed: 0')

In [9]:
pone.head()

,Year,Title,Abstract,Role,Affiliation index,Affiliation list,Discipline,Filename
0,2016,"1,25(OH)2D3 and VDR Signaling Pathways Regulat...",\n\nBackground\nThe objective of this study is...,"[[], [], [], [], [], [], [], []]","[['aff001'], ['aff001', 'cor001'], ['aff001'],...","['1 Department of Ophthalmology, The Affiliate...","['Medicine and health sciences', 'Anatomy', 'B...",/data02/plos/pone/journal.pone.0164717.xml
1,2018,"A non-invasive, quantitative study of broadban...","\nCurrently, non-invasive methods for studying...","[['Data curation', 'Formal analysis', 'Investi...","[['aff001', 'cor001'], ['aff001'], ['aff002'],...",['1 Department of Psychology and Center for Ne...,"['Engineering and technology', 'Signal process...",/data02/plos/pone/journal.pone.0193107.xml
2,2016,The Effects of Financial Education on Impulsiv...,"\nDelay discounting, as a behavioral measure o...","[[], [], [], []]","[['aff001', 'cor001'], ['aff001'], ['aff002'],...","['1 Department of Psychology, Utah State Unive...","['Social sciences', 'Psychology', 'Personality...",/data02/plos/pone/journal.pone.0159561.xml
3,2016,Root-Zone Warming Differently Benefits Mature ...,\nSub-optimal temperature extensively suppress...,"[[], [], [], []]","[['aff001'], ['aff001'], ['aff001'], ['cor001'...",['Beijing Key Laboratory of Growth and Develop...,"['Medicine and health sciences', 'Vascular med...",/data02/plos/pone/journal.pone.0155298.xml
4,2017,A hypomorphic PIGA gene mutation causes severe...,\nMutations in genes involved in glycosylphosp...,"[[], [], [], [], [], [], [], [], [], [], []]","[['aff001'], ['aff002'], ['aff001'], ['aff001'...","['1 Division of Hematology, Department of Medi...","['Biology and life sciences', 'Cell biology', ...",/data02/plos/pone/journal.pone.0174074.xml


In [10]:
pone['Abstract']= pone['Abstract'].apply(lambda x: " ".join(x.split()))
pone['Title']= pone['Title'].apply(lambda x: " ".join(x.split()))

In [11]:
pone['Total'] = pone['Abstract']+pone['Title']

## Three sentence

In [12]:
import numpy as np

In [16]:
text_query = "Chemical hydrogels are commonly prepared in two different ways: 'three-dimensional polymerization' (Fig. 1), in which a hydrophilic monomer is polymerized in the presence of a polyfunctional cross-linking agent, or by direct cross-linking of water-soluble polymers (Fig. 2)." 
    
    
    

In [17]:
text_A = "Hydrogels can be obtained by radiation technique in a few ways, including irradiation of solid polymer, monomer (in bulk or in solution), or aqueous solution of polymer."
    

In [18]:
  text_B = "The observed cosmic shear auto-power spectrum receives an additional contribution due to shape noise from intrinsic galaxy ellipticities."
    
    

In [19]:
query_emb = get_scibert_embedding(text_query)[1][0]
query_sentence_emb =  np.mean(query_emb,axis=0)

textA_emb = get_scibert_embedding(text_A)[1][0]
textA_sentence_emb =  np.mean(textA_emb,axis=0)

textB_emb = get_scibert_embedding(text_B)[1][0]
textB_sentence_emb =  np.mean(textB_emb,axis=0)

In [13]:
from scipy.spatial import distance

In [21]:
1- distance.cosine(query_sentence_emb,textA_sentence_emb)

0.9608220458030701

In [22]:
1-distance.cosine(query_sentence_emb,textB_sentence_emb)

0.8867639303207397

In [23]:
text= "The man in prison watched the animal from his cell"
scibert_id, scibert_emb = get_scibert_embedding(text)


In [24]:
cell_emb = scibert_emb[scibert_id == scibert_tokenizer.vocab['cell']]
animal_emb = scibert_emb[scibert_id == scibert_tokenizer.vocab['animal']]

prison_emb = scibert_emb[scibert_id == scibert_tokenizer.vocab['prison']]



In [25]:
print('    sim)cell, animal): {:.2}'.format(1-distance.cosine(cell_emb,animal_emb)))

    sim)cell, animal): 0.82


# Learning BERT on Pone

In [30]:
pone['Total_trained'] = pone['Total'].apply(lambda x: np.mean(get_scibert_embedding(x)[1][0],axis=0))

KeyboardInterrupt: 

In [14]:
from tqdm import tqdm

In [16]:
Total_train = []
for total in tqdm(pone['Total']):
    Total_train.append(np.mean(get_bert_embedding(total)[1][0],axis=0))

  0%|          | 26/70387 [00:00<24:26, 47.96it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors
  0%|          | 79/70387 [00:01<21:27, 54.62it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors
  0%|          | 115/70387 [00:02<22:16, 52.59it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running th

In [19]:
pone['Total_trained'] = list(Total_train)

In [34]:
standard = pone['Total_trained'][32682]

In [35]:
standard_cos = pone['Total_trained'].apply(lambda x: 1-distance.cosine(x,standard))

In [43]:
np.save('../../Data/Experiment/Plosone/pone_sciBERT',np.array(Total_train))

In [39]:
np.save('../../Data/Experiment/Plosone/pone_BERT',np.array(Total_train))

In [12]:
Total_train =np.load('../../Data/Experiment/Plosone/pone_BERT.csv.npy',allow_pickle=True)

In [36]:
standard_cos.sort_values(ascending=False).index[:10]

Int64Index([32682, 64076, 8374, 46216, 65256, 30440, 22404, 41591, 29941,
            43633],
           dtype='int64')

In [37]:
pone['Title'][standard_cos.sort_values(ascending=False).index[:10]]

32682    Measuring national capability over big science...
64076    International Collaboration Patterns and Effec...
8374     How Does National Scientific Funding Support E...
46216    Scientific Wealth in Middle East and North Afr...
65256    A case against the trickle-down effect in tech...
30440    Visualized analysis of developing trends and h...
22404    Measuring popularity of ecological topics in a...
41591    Examination of China’s performance and themati...
29941    Global impact of accelerated plant breeding: E...
43633    Renewable energy consumption and economic grow...
Name: Title, dtype: object

In [40]:
print(pone['Discipline'][])
print(pone['Title'][2])

['Social sciences', 'Psychology', 'Personality', 'Personality differences']
The Effects of Financial Education on Impulsive Decision Making


In [54]:
standard_cos.sort_values(ascending=False)

2        1.000000
22687    0.961219
27648    0.961184
42548    0.960629
49888    0.960509
           ...   
65842    0.677452
53498    0.676423
33722    0.672093
68803    0.671758
30520    0.661946
Name: Total_trained, Length: 70387, dtype: float64

In [38]:
for i in range(10):
    i = standard_cos.sort_values(ascending=False).index[i]
    print(pone['Discipline'][i])
    print(pone['Title'][i]+'\n')
    print(standard_cos.sort_values(ascending=False)[i])
    print("==================================")

['People and places', 'Geographical locations', 'Asia', 'India']
Measuring national capability over big science’s multidisciplinarity: A case study of nuclear fusion research

1.0
['Social sciences', 'Economics', 'Economic analysis']
International Collaboration Patterns and Effecting Factors of Emerging Technologies

0.941048800945282
['People and places', 'Population groupings', 'Professions', 'Engineers']
How Does National Scientific Funding Support Emerging Interdisciplinary Research: A Comparison Study of Big Data Research in the US and China

0.9362369775772095
['Engineering and technology', 'Geological engineering']
Scientific Wealth in Middle East and North Africa: Productivity, Indigeneity, and Specialty in 1981–2013

0.9340124130249023
['Research and analysis methods', 'Mathematical and statistical techniques', 'Mathematical functions']
A case against the trickle-down effect in technology ecosystems

0.9301679730415344
['Engineering and technology']
Visualized analysis of deve

In [27]:
for i in range(10):
    i = standard_cos.sort_values(ascending=False).index[i]
    print(pone['Discipline'][i])
    print(pone['Title'][i]+'\n')
    print(standard_cos.sort_values(ascending=False)[i])
    print("==============================================================")

['Social sciences', 'Psychology', 'Personality', 'Personality differences']
The Effects of Financial Education on Impulsive Decision Making

1.0
['Medicine and health sciences', 'Mental health and psychiatry', 'Mood disorders', 'Depression']
Predictors of Participation of Sophomore Medical Students in a Health-Promoting Intervention: An Observational Study

0.949718177318573
['Physical sciences', 'Mathematics', 'Statistics', 'Statistical methods', 'Metaanalysis']
Common mental disorders and subjective well-being: Emotional training among medical students based on positive psychology

0.9441468119621277
['Biology and life sciences', 'Neuroscience', 'Learning and memory', 'Memory']
Brain training improves recovery after stroke but waiting list improves equally: A multicenter randomized controlled trial of a computer-based cognitive flexibility training

0.9415279030799866
['Research and analysis methods', 'Research design', 'Longitudinal studies']
Long-term effects of an e-learning cours

In [33]:
pone[pone['Filename'].apply(lambda x: True if '0211963' in x else False)]

,Year,Title,Abstract,Role,Affiliation index,Affiliation list,Discipline,Filename,Total,Total_trained
32682,2019,Measuring national capability over big science...,"In the era of big science, countries allocate ...","[['Formal analysis', 'Writing – original draft...","[['aff001', 'aff002', 'aff003'], ['aff004'], [...",['\n1\nDepartment of Industrial and Management...,"['People and places', 'Geographical locations'...",/data02/plos/pone/journal.pone.0211963.xml,"In the era of big science, countries allocate ...","[-0.35765022, 0.1548143, 0.24890673, 0.0701792..."


In [ ]:
0211963 